In [1]:
import pandas as pd
import pymongo
import csv

In [2]:
client = pymongo.MongoClient("localhost:27017")
db = client["Airbnb"]

In [3]:
def getExecutionTimeBySecond(cursor):
    return str(cursor.explain()['executionStats']['executionTimeMillis']/1000) + ' sec'

In [4]:
listing = db['Listing'].find_one({"name": "Nice room with superb city view"})
availability = db['Calendar'].find({"property": listing['_id'], "date": "2022-07-20"}, {"availability": 1})

print(list(availability))
print(getExecutionTimeBySecond(availability))

[{'_id': ObjectId('636b583c69c65a2eb1f7fec5'), 'availability': True}]
2.765 sec


In [5]:
update = db.Calendar.update_many({"property": listing['_id'], "date": {"$gte": "2022-06-21", "$lt": "2022-06-22"}}, {"$set": {"availability": False}})
update.raw_result

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

In [6]:
listings = db.Listing.find({"neighbourhood": "Sathon", "bedrooms": 1, "beds": 1, "accommodates": 2, "review_scores_rating": {"$gte": 4}})

print('records length:' + str(len(list(listings))))
print(getExecutionTimeBySecond(listings))

records length:120
0.014 sec


In [7]:
update = db.Transaction.update_one({"_id": 180261}, {"$set": {"review":"Nice!!"}})
update.raw_result

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

In [8]:
hosts = db.Listing.find({"review_scores_rating": {"$gte": 4}}, {"host": 1})
hosts = db.Host.find({"$and": [{ "_id": {"$in": [h['host'] for h in list(hosts)]}}, 
                               {"response_rate": {"$gte": 1}}]
                     }, {'_id': 1})

print('records length:' + str(len(list(hosts))))
print(getExecutionTimeBySecond(hosts))

records length:1825
0.012 sec


In [9]:
customers = db.Customer.find({"name": "Tom"})

print('records length:' + str(len(list(customers))))
print(getExecutionTimeBySecond(customers))

records length:289
0.084 sec


In [10]:
transactions = db.Transaction.find({"property": 39544047}, \
                                      {"customer": 1, "property": 1, "start_date": 1, "end_date": 1, "total_price": 1, "review": 1}) \
                                .sort("start_date", -1)
transactions = list(transactions)

customers = db.Customer.find({"_id": {"$in": [int(t['customer']) for t in transactions]}})
customers = list(customers)

customers

[{'_id': 19143977, 'name': 'Ploi'},
 {'_id': 143251478, 'name': 'Oliver'},
 {'_id': 163790201, 'name': 'Yang'},
 {'_id': 273940743, 'name': 'Kim'},
 {'_id': 288635322, 'name': 'Doris'},
 {'_id': 317920443, 'name': 'Kunat'}]

In [11]:
transactions = db.Transaction.find({"property": 39544047}, \
                                      {"customer": 1, "property": 1, "start_date": 1, "end_date": 1, "total_price": 1, "review": 1}) \
                                .sort("start_date", -1)

listing = db['Listing'].find_one({"name": "Nice room with superb city view"})
list(db.Listing.find({"_id": {"$in": [int(t['property']) for t in transactions]}}, {"review_scores_rating": 1}))

[{'_id': 39544047, 'review_scores_rating': 4.67}]

In [12]:
list(db.Host.find({"_id": listing['host']}, {"name": 1, "description": 1, "response_rate": 1, "response_time": 1, "is_super_host": 1, "email_verified": 1, "phone_verified": 1, "identity_verified": 1, "url": 1}))

[{'_id': 120437,
  'name': 'Nuttee',
  'description': 'Hi All, I am nuttee patranavik from Bangkok, Thailand.always travel but easy to connect via airbnb..',
  'response_time': nan,
  'response_rate': 0.0,
  'is_super_host': False,
  'email_verified': True,
  'phone_verified': True,
  'identity_verified': True,
  'url': 'https://www.airbnb.com/users/show/120437'}]